In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
import numpy as np
from matplotlib import cm
import datetime
from datetime import datetime
import seaborn as sns
from scipy import stats
import uuid



#raw = pd.pandas.read_csv('../setup_data/pred_1x1_20181206.csv',sep=",",low_memory=False)
raw = pd.pandas.read_csv('../20181220_raw_geo.csv',sep=",",low_memory=False)

#https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

# Setup Data Set for variable calculation

In [36]:
# copy raw data & create new base variables
data = raw.copy()
data['published_dt'] = pd.to_datetime(raw['published'])
data['sold_dt'] = pd.to_datetime(raw['sold_date'])
data['sqm_price_diff'] = data['sqm_sold_price'] - data['sqm_list_price']
data['cnt'] = 1
data = data[data['published'] >'2018-09-01']
data['sqm_bin'],sqm_bins_list = pd.qcut(data['living_area'], 5,retbins=True,labels=) # generate sqm buckets
data['uuid'] = [uuid.uuid4() for _ in range(len(data))]

# choose vars needed for calculation
# set important indexes
print(len(data))

TypeError: object of type 'bool' has no len()

In [34]:
data['sqm_bin']

2893       (51.5, 64.0]
2895       (51.5, 64.0]
5619     (80.88, 337.0]
5621     (80.88, 337.0]
5645       (51.5, 64.0]
5647       (51.5, 64.0]
5667       (51.5, 64.0]
5669       (51.5, 64.0]
5686     (14.999, 39.0]
5688     (14.999, 39.0]
5876      (64.0, 80.88]
5878      (64.0, 80.88]
5986      (64.0, 80.88]
5988      (64.0, 80.88]
6367     (80.88, 337.0]
6369     (80.88, 337.0]
7342       (51.5, 64.0]
7344       (51.5, 64.0]
7479       (51.5, 64.0]
7481       (51.5, 64.0]
8171       (51.5, 64.0]
8211     (14.999, 39.0]
8213       (39.0, 51.5]
8233       (39.0, 51.5]
8235     (80.88, 337.0]
8239     (14.999, 39.0]
8244      (64.0, 80.88]
8246       (51.5, 64.0]
8250      (64.0, 80.88]
8254       (51.5, 64.0]
              ...      
70452    (14.999, 39.0]
70453      (39.0, 51.5]
70454      (51.5, 64.0]
70455      (39.0, 51.5]
70456     (64.0, 80.88]
70457      (51.5, 64.0]
70458      (39.0, 51.5]
70459      (51.5, 64.0]
70460    (14.999, 39.0]
70461    (80.88, 337.0]
70462      (39.0

In [26]:
#######################################################################
## Define functions that will be used in lambda functions for a dataset
#######################################################################

# Expanding window mean for a variable
def expanding_mean_for_var(row,var_to_calc,historic_df):
    var = historic_df[
        (historic_df.index.get_level_values('published_dt') < row.name)
    ][var_to_calc].mean()
    return var

# Expanding window mean for a variable in a group
def expanding_mean_for_var_per_group(row,var_to_calc,historic_df,group):
    var = historic_df[
        (historic_df.index.get_level_values('published_dt') < row.name)
        & (historic_df.index.get_level_values('area_from_geo') == row.name[1])
    ][var_to_calc].mean()
    return var


def rolling_mean_for_var(row,var_to_calc,historic_df,days):
    var = historic_df[
       (historic_df.index.get_level_values('published_dt') < row.name)
       & (historic_df.index.get_level_values('published_dt') > (row.name - pd.to_timedelta(days, unit='d')))
    ][var_to_calc].mean()
    return var


# Rolling window mean for a variable in a group
def rolling_mean_for_var_per_group(row,var_to_calc,historic_df,group,days):
    var = historic_df[
        (historic_df.index.get_level_values('published_dt') < row.name[0])
       & (historic_df.index.get_level_values('published_dt') > (row.name[0] - pd.to_timedelta(days, unit='d')))
        & (historic_df.index.get_level_values(group) == row.name[1])
    ][var_to_calc].mean()
    return var


# Calculate variables without group

In [27]:
# Setup new dataset for purpose
df_no_group = data[['uuid','published_dt','sqm_sold_price','sqm_price_diff','living_area','sqm_rent_price']].copy()
df_no_group = df_no_group.set_index('published_dt')

before = datetime.now()
rolling_mean_sqm_price_90d = df_no_group.apply(lambda row: rolling_mean_for_var(row=row,var_to_calc='sqm_sold_price',historic_df=df_no_group,days=90),axis=1)
rolling_mean_sqm_price_diff_90d = df_no_group.apply(lambda row: rolling_mean_for_var(row=row,var_to_calc='sqm_price_diff',historic_df=df_no_group,days=90),axis=1)
rolling_mean_sqm_90d = df_no_group.apply(lambda row: rolling_mean_for_var(row=row,var_to_calc='living_area',historic_df=df_no_group,days=90),axis=1)
rolling_mean_sqm_rent_90d = df_no_group.apply(lambda row: rolling_mean_for_var(row=row,var_to_calc='sqm_rent_price',historic_df=df_no_group,days=90),axis=1)

after = datetime.now()
print("run time: "+str((after-before).total_seconds()) + " sec")



run time: 21.071618 sec


# Calculate variables with groups

In [6]:
## AREA FROM GEO
# Setup new dataset for purpose
df_group_area = data[['uuid','published_dt','sqm_sold_price','sqm_price_diff','living_area','sqm_rent_price','area_from_geo']].copy()
df_group_area = df_group_area.set_index(['published_dt','area_from_geo'])

# only calculate for apts where more than top 20% apts in area
geo_cnt_limit = 0.05*max(df_group_area.groupby('area_from_geo').sqm_sold_price.transform(len))
df_group_area = df_group_area[df_group_area.groupby('area_from_geo').sqm_sold_price.transform(len) > geo_cnt_limit]

before = datetime.now()
df_group_area['rolling_mean_sqm_price_per_area_90d'] = df_group_area.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_sold_price',historic_df=df_group_area,group='area_from_geo',days=90),axis=1)
df_group_area['rolling_mean_sqm_price_diff_per_area_90d'] = df_group_area.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_price_diff',historic_df=df_group_area,group='area_from_geo',days=90),axis=1)
df_group_area['rolling_mean_sqm_per_area_90d'] = df_group_area.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='living_area',historic_df=df_group_area,group='area_from_geo',days=90),axis=1)
df_group_area['rolling_mean_sqm_rent_per_area_90d'] = df_group_area.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_rent_price',historic_df=df_group_area,group='area_from_geo',days=90),axis=1)

df_group_area = df_group_area.reset_index(drop=True)[['uuid','rolling_mean_sqm_price_per_area_90d','rolling_mean_sqm_price_diff_per_area_90d','rolling_mean_sqm_per_area_90d','rolling_mean_sqm_rent_per_area_90d']]
df_group_area = df_group_area.set_index('uuid')
after = datetime.now()
print("run time: "+str((after-before).total_seconds()) + " sec")


run time: 27.438861 sec


In [28]:
## SQM BIN
# Setup new dataset for purpose
df_group_sqm_bin = data[['uuid','published_dt','sqm_sold_price','sqm_price_diff','living_area','sqm_rent_price','sqm_bin']].copy()
df_group_sqm_bin = df_group_sqm_bin.set_index(['published_dt','sqm_bin'])

# only calculate for apts where more than top 20% apts in area
geo_cnt_limit = 0.05*max(df_group_sqm_bin.groupby('sqm_bin').sqm_sold_price.transform(len))
df_group_sqm_bin = df_group_sqm_bin[df_group_sqm_bin.groupby('sqm_bin').sqm_sold_price.transform(len) > geo_cnt_limit]

before = datetime.now()
df_group_sqm_bin['rolling_mean_sqm_price_per_sqm_bin_90d'] = df_group_sqm_bin.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_sold_price',historic_df=df_group_sqm_bin,group='sqm_bin',days=90),axis=1)
df_group_sqm_bin['rolling_mean_sqm_price_diff_per_sqm_bin_90d'] = df_group_sqm_bin.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_price_diff',historic_df=df_group_sqm_bin,group='sqm_bin',days=90),axis=1)
df_group_sqm_bin['rolling_mean_sqm_per_sqm_bin_90d'] = df_group_sqm_bin.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='living_area',historic_df=df_group_sqm_bin,group='sqm_bin',days=90),axis=1)
df_group_sqm_bin['rolling_mean_sqm_rent_sqm_bin_90d'] = df_group_sqm_bin.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_rent_price',historic_df=df_group_sqm_bin,group='sqm_bin',days=90),axis=1)

df_group_sqm_bin = df_group_sqm_bin.reset_index(drop=True)[['uuid','rolling_mean_sqm_price_per_sqm_bin_90d','rolling_mean_sqm_price_diff_per_sqm_bin_90d','rolling_mean_sqm_per_sqm_bin_90d','rolling_mean_sqm_rent_sqm_bin_90d']]
df_group_sqm_bin = df_group_sqm_bin.set_index('uuid')
after = datetime.now()
print("run time: "+str((after-before).total_seconds()) + " sec")

run time: 31.122134 sec


In [8]:
## BROKER
# Setup new dataset for purpose
df_group_broker = data[['uuid','published_dt','sqm_sold_price','sqm_price_diff','living_area','sqm_rent_price','source_name']].copy()
df_group_broker = df_group_broker.set_index(['published_dt','source_name'])

# only calculate for apts where more than top 20% apts in area
geo_cnt_limit = 0.05*max(df_group_broker.groupby('source_name').sqm_sold_price.transform(len))
df_group_broker = df_group_broker[df_group_broker.groupby('source_name').sqm_sold_price.transform(len) > geo_cnt_limit]

before = datetime.now()
df_group_broker['rolling_mean_sqm_price_per_broker_90d'] = df_group_broker.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_sold_price',historic_df=df_group_broker,group='source_name',days=90),axis=1)
df_group_broker['rolling_mean_sqm_price_diff_per_broker_90d'] = df_group_broker.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_price_diff',historic_df=df_group_broker,group='source_name',days=90),axis=1)
df_group_broker['rolling_mean_sqm_per_broker_90d'] = df_group_broker.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='living_area',historic_df=df_group_broker,group='source_name',days=90),axis=1)
df_group_broker['rolling_mean_sqm_rent_per_broker_90d'] = df_group_broker.apply(lambda row: rolling_mean_for_var_per_group(row=row,var_to_calc='sqm_rent_price',historic_df=df_group_broker,group='source_name',days=90),axis=1)

df_group_broker = df_group_broker.reset_index(drop=True)[['uuid','rolling_mean_sqm_price_per_broker_90d','rolling_mean_sqm_price_diff_per_broker_90d','rolling_mean_sqm_per_broker_90d','rolling_mean_sqm_rent_per_broker_90d']]
df_group_broker = df_group_broker.set_index('uuid')
after = datetime.now()
print("run time: "+str((after-before).total_seconds()) + " sec")

run time: 29.91754 sec


In [9]:
output = data.copy()
output = pd.merge(output,df_group_area,left_on = 'uuid', right_index = True, how = 'left')
output = pd.merge(output,df_group_sqm_bin,left_on = 'uuid', right_index = True, how = 'left')
output = pd.merge(output,df_group_broker,left_on = 'uuid', right_index = True, how = 'left')

# Save Output

In [12]:
output = output.loc[data['published_dt'] >'2016-04-01'][
    ['published_dt'
    ,'sqm_sold_price'
    ,'rooms'
    ,'floor'
    ,'rent'
    ,'living_area'
    ,'construction_year'
    ,'distance_ocean'
    ,'published_week'
    ,'rolling_mean_sqm_price_diff_per_area_90d'
    ,'rolling_mean_sqm_per_area_90d'
    ,'rolling_mean_sqm_rent_per_area_90d'
    ,'rolling_mean_sqm_price_per_sqm_bin_90d'
    ,'rolling_mean_sqm_price_diff_per_sqm_bin_90d'
    ,'rolling_mean_sqm_per_sqm_bin_90d'
    ,'rolling_mean_sqm_rent_sqm_bin_90d'
    ,'rolling_mean_sqm_price_per_broker_90d'
    ,'rolling_mean_sqm_price_diff_per_broker_90d'
    ,'rolling_mean_sqm_per_broker_90d'
    ,'rolling_mean_sqm_rent_per_broker_90d'
    ]
]
output.to_csv('featured_variables.csv',index=False)
